In [1]:
import numpy as np
import networkx as nx
from networkx.generators.community import LFR_benchmark_graph, planted_partition_graph,random_partition_graph
from networkx.algorithms import community
import os
import math

import functools

import seaborn as sns
import pandas as pd
from matplotlib import pyplot as plt
import numpy as np
import re
from itertools import islice

In [72]:
dic = {}
with open('/home/sfy/Documents/testSNA/SNAPro/dataset/email-Eu-core-department-labels.txt') as file:
    for line in file.readlines():
        line = line.strip().split(' ')
        node = line[0]
        comm = line[1]
        if dic.get(comm)==None:
            dic.setdefault(comm,'')
            dic[comm] = node
        else:
            dic[comm] = dic[comm] + ' '+ node

In [75]:
dic['1']

'0 1 17 18 73 74 85 120 177 215 218 219 220 221 222 223 224 225 226 227 228 248 297 307 308 309 310 311 312 313 314 315 316 317 341 414 415 449 459 468 479 516 517 603 605 629 645 650 675 696 697 731 734 743 764 775 797 798 846 852 872 904 905 916 1002'

In [76]:
with open('testout.txt','w') as file:
        for comms in dic.values():
            file.write(comms+'\n')
    

In [ ]:
seed_set = [2, 34,56]